<center><img src="../../图片数据/logo.png" alt="Header" style="width: 800px;"/></center>

@Copyright (C): 2010-2019, Shenzhen Yahboom Tech  
@Author: Malloy.Yuan  
@Date: 2019-07-17 10:10:02  
@LastEditors: Malloy.Yuan  
@LastEditTime: 2019-09-17 17:54:19  

### 创建机器人实例
我们通过使用Robot对象调用我们已经包装好的电机驱动库驱动电机

In [ ]:
from jetbot import Robot
import time
robot = Robot()

### GPIO口初始化配置
从RPi.GPIO导入GPIO对象,初始化引脚对象
将限位开关的GPIO口模式设置为BCM模式,设置为输入模式

In [ ]:
import RPi.GPIO as GPIO
up_limit_pin = 17
down_limit_pin = 18
GPIO.setmode(GPIO.BCM)
GPIO.setup(up_limit_pin, GPIO.IN)
GPIO.setup(down_limit_pin, GPIO.IN)

### 导入python线程库,初始化云台升降变量
创建vertical_motors_action云台升降全局变量,并导入线程包监控云台升降状态

In [ ]:
import threading
global vertical_motors_action
vertical_motors_action = 0

### 云台升降上下限检测方法

In [ ]:
def limit_detect():
    global vertical_motors_action
    while 1:
        if vertical_motors_action == 1:
            if GPIO.input(up_limit_pin) == 0:
                robot.vertical_motors_stop()
                vertical_motors_action = 0
                print('云台到顶')
        elif vertical_motors_action == 2:
            if GPIO.input(down_limit_pin) == 0:
                robot.vertical_motors_stop()
                vertical_motors_action = 0
                print('云台到底')
        time.sleep(0.5)

### 新建并开启监控云台上下限的线程

In [ ]:
thread1 = threading.Thread(target=limit_detect)
# thread1.setDaemon(True)
thread1.start()

### 云台上升的方法

In [ ]:
vertical_motors_action = 1
if(GPIO.input(up_limit_pin)):
    robot.up(1)
    print('cameraup')
else:
    print('云台到顶')
    robot.vertical_motors_stop()
    vertical_motors_action = 0

### 云台下降的方法

In [ ]:
vertical_motors_action = 2
if(GPIO.input(down_limit_pin)):
    robot.down(1)
    print('cameradown')
else:
    print('云台到底')
    robot.vertical_motors_stop()
    vertical_motors_action = 0

### 云台停止的方法

In [ ]:
robot.vertical_motors_stop()
vertical_motors_action = 0
print('camerastop')